# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgbm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('/datasets/autos.csv')

In [3]:
print(data.isna().mean())
print(data.corr())
print(data.describe())

DateCrawled          0.000000
Price                0.000000
VehicleType          0.105794
RegistrationYear     0.000000
Gearbox              0.055967
Power                0.000000
Model                0.055606
Kilometer            0.000000
RegistrationMonth    0.000000
FuelType             0.092827
Brand                0.000000
NotRepaired          0.200791
DateCreated          0.000000
NumberOfPictures     0.000000
PostalCode           0.000000
LastSeen             0.000000
dtype: float64
                      Price  RegistrationYear     Power  Kilometer  \
Price              1.000000          0.026916  0.158872  -0.333199   
RegistrationYear   0.026916          1.000000 -0.000828  -0.053447   
Power              0.158872         -0.000828  1.000000   0.024002   
Kilometer         -0.333199         -0.053447  0.024002   1.000000   
RegistrationMonth  0.110581         -0.011619  0.043380   0.009571   
NumberOfPictures        NaN               NaN       NaN        NaN   
PostalCode     

In [4]:
data

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354364,2016-03-21 09:50:58,0,NaN,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes,2016-03-21 00:00:00,0,2694,2016-03-21 10:42:49
354365,2016-03-14 17:48:27,2200,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,2016-03-14 00:00:00,0,39576,2016-04-06 00:46:52
354366,2016-03-05 19:56:21,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no,2016-03-05 00:00:00,0,26135,2016-03-11 18:17:12
354367,2016-03-19 18:57:12,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no,2016-03-19 00:00:00,0,87439,2016-04-07 07:15:26


Загружен исходный датасет. Данные имеют пропуски в столбцах  VehicleType, Gearbox, Model, FuelType, NotRepaired. В столбцах Price  и  Power присутствуют нулевые значения. Кроме того в столбце RegistrationYear указаны неверные годы регистрации. Изучем уникальные признаки из столбцов с пропусками и постараемся заменить их подходящими значениями.

In [5]:
data['VehicleType'].unique()

array([nan, 'coupe', 'suv', 'small', 'sedan', 'convertible', 'bus',
       'wagon', 'other'], dtype=object)

In [6]:
data['Gearbox'].unique()

array(['manual', 'auto', nan], dtype=object)

In [7]:
data['FuelType'].unique()

array(['petrol', 'gasoline', nan, 'lpg', 'other', 'hybrid', 'cng',
       'electric'], dtype=object)

In [8]:
data['Brand'].unique()

array(['volkswagen', 'audi', 'jeep', 'skoda', 'bmw', 'peugeot', 'ford',
       'mazda', 'nissan', 'renault', 'mercedes_benz', 'opel', 'seat',
       'citroen', 'honda', 'fiat', 'mini', 'smart', 'hyundai',
       'sonstige_autos', 'alfa_romeo', 'subaru', 'volvo', 'mitsubishi',
       'kia', 'suzuki', 'lancia', 'toyota', 'chevrolet', 'dacia',
       'daihatsu', 'trabant', 'saab', 'chrysler', 'jaguar', 'daewoo',
       'porsche', 'rover', 'land_rover', 'lada'], dtype=object)

In [9]:
data[data['RegistrationYear']<1980]

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
15,2016-03-11 21:39:15,450,small,1910,NaN,0,ka,5000,0,petrol,ford,NaN,2016-03-11 00:00:00,0,24148,2016-03-19 08:46:47
360,2016-03-17 14:52:07,10000,bus,1976,NaN,0,transporter,5000,0,petrol,volkswagen,NaN,2016-03-17 00:00:00,0,73479,2016-03-19 14:21:08
534,2016-03-27 10:52:55,4900,convertible,1973,manual,44,kaefer,90000,4,petrol,volkswagen,no,2016-03-27 00:00:00,0,16567,2016-04-07 06:44:31
622,2016-03-16 16:55:09,0,NaN,1111,NaN,0,NaN,5000,0,NaN,opel,NaN,2016-03-16 00:00:00,0,44628,2016-03-20 16:44:37
630,2016-03-18 12:43:41,12000,sedan,1969,manual,140,s_klasse,80000,3,petrol,mercedes_benz,no,2016-03-18 00:00:00,0,46286,2016-04-05 22:16:44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353961,2016-03-17 13:54:22,200,NaN,1910,NaN,0,NaN,5000,0,petrol,sonstige_autos,NaN,2016-03-17 00:00:00,0,42289,2016-03-31 22:46:47
354037,2016-03-23 16:57:30,10500,convertible,1968,manual,54,other,20000,4,NaN,renault,no,2016-03-23 00:00:00,0,63755,2016-03-25 00:47:18
354063,2016-03-08 23:37:04,0,NaN,1970,NaN,0,sl,150000,0,NaN,mercedes_benz,NaN,2016-03-08 00:00:00,0,34123,2016-04-07 05:15:38
354286,2016-03-21 20:44:38,16500,coupe,1973,manual,200,other,150000,6,petrol,bmw,yes,2016-03-21 00:00:00,0,74722,2016-04-06 19:16:32


In [10]:
# удалим строки с пропусками в поле Model
data.dropna(subset=['Model'], inplace = True)

# заменим пропуски в полях 'FuelType' 'Gearbox' 'VehicleType' на характерные для определенной модели автомобиля
data['FuelType'].fillna(data.groupby('Model')['FuelType'].transform('first'), inplace=True)
data['Gearbox'].fillna(data.groupby('Model')['Gearbox'].transform('first'), inplace=True)
data['VehicleType'].fillna(data.groupby('Model')['VehicleType'].transform('first'), inplace=True)

# поставим ограничение по мощности автомобиля. приравняем выбивающиеся значения к nan и заменим 
#их медианным показателем аналогичной модели. несопоставленные значения удалим
data.loc[data['Power']>200, 'Power']=0
data.loc[data['Power']<50, 'Power']=0
data['Power'] = data['Power'].replace(0, np.nan)
data['Power'].fillna(data.groupby('Model')['Power'].transform('median'), inplace=True)
data.dropna(subset=['Power'], inplace = True)

# заменим аналоги в поле FuelType на одно из значений. значение other возмем аналогичным для конкретной модели автомобиля
data['FuelType'] = data['FuelType'].replace('cng', 'lpg')
data['FuelType'] = data['FuelType'].replace('gasoline', 'petrol')
data['FuelType'] = data['FuelType'].replace('other', np.nan)
data['FuelType'].fillna(data.groupby('Model')['FuelType'].transform('first'), inplace=True)

# приравняем выбивающися значения даты регистрации к nan и заменим их минимальным значением для данной модели автомобиля.
# несопоставленные значения удалим. Изменими тип данных в столбце Дата регистрации.
data.loc[data['RegistrationYear']>2016, 'RegistrationYear']=0
data.loc[data['RegistrationYear']<1990, 'RegistrationYear']=0
data['RegistrationYear'] = data['RegistrationYear'].replace(0, np.nan)
data['RegistrationYear'].fillna(data.groupby('Model')['RegistrationYear'].transform('max'), inplace=True)
data.dropna(subset=['RegistrationYear'], inplace = True)
data['RegistrationYear']=data['RegistrationYear'].astype('int64')

In [11]:
print(data.isna().mean())

DateCrawled          0.000000
Price                0.000000
VehicleType          0.000000
RegistrationYear     0.000000
Gearbox              0.000000
Power                0.000000
Model                0.000000
Kilometer            0.000000
RegistrationMonth    0.000000
FuelType             0.000000
Brand                0.000000
NotRepaired          0.185671
DateCreated          0.000000
NumberOfPictures     0.000000
PostalCode           0.000000
LastSeen             0.000000
dtype: float64


пропуски из ключевых столбцов удалены

In [12]:
data

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,small,1993,manual,101.0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
2,2016-03-14 12:52:21,9800,suv,2004,auto,163.0,grand,125000,8,petrol,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75.0,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69.0,fabia,90000,7,petrol,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
5,2016-04-04 17:36:23,650,sedan,1995,manual,102.0,3er,150000,10,petrol,bmw,yes,2016-04-04 00:00:00,0,33775,2016-04-06 19:17:07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354363,2016-03-27 20:36:20,1150,bus,2000,manual,125.0,zafira,150000,3,petrol,opel,no,2016-03-27 00:00:00,0,26624,2016-03-29 10:17:23
354364,2016-03-21 09:50:58,0,sedan,2005,manual,82.0,colt,150000,7,petrol,mitsubishi,yes,2016-03-21 00:00:00,0,2694,2016-03-21 10:42:49
354366,2016-03-05 19:56:21,1199,convertible,2000,auto,101.0,fortwo,125000,3,petrol,smart,no,2016-03-05 00:00:00,0,26135,2016-03-11 18:17:12
354367,2016-03-19 18:57:12,9200,bus,1996,manual,102.0,transporter,150000,3,petrol,volkswagen,no,2016-03-19 00:00:00,0,87439,2016-04-07 07:15:26


In [13]:
print(data.describe())

               Price  RegistrationYear          Power      Kilometer  \
count  334198.000000     334198.000000  334198.000000  334198.000000   
mean     4491.148783       2003.994773     112.706895  128555.167894   
std      4518.120226          6.219033      38.717370   37211.258311   
min         0.000000       1990.000000      50.000000    5000.000000   
25%      1150.000000       1999.000000      75.000000  125000.000000   
50%      2800.000000       2004.000000     110.000000  150000.000000   
75%      6500.000000       2008.000000     143.000000  150000.000000   
max     20000.000000       2016.000000     200.000000  150000.000000   

       RegistrationMonth  NumberOfPictures     PostalCode  
count      334198.000000          334198.0  334198.000000  
mean            5.804876               0.0   50671.456502  
std             3.688998               0.0   25755.436189  
min             0.000000               0.0    1067.000000  
25%             3.000000               0.0   30419.

In [14]:
# уберем экстремально малые значения целевого признака
price_min_threshold = data['Price'].quantile(0.05)
price_max_threshold = data['Price'].quantile(0.95)
data_filtered = data[data['Price']>=price_min_threshold]

In [15]:
print(data_filtered.describe())
print(data_filtered.corr())

               Price  RegistrationYear          Power      Kilometer  \
count  317610.000000     317610.000000  317610.000000  317610.000000   
mean     4722.395176       2004.125021     113.500893  128574.730015   
std      4516.839938          6.111377      38.621834   36768.064400   
min       250.000000       1990.000000      50.000000    5000.000000   
25%      1300.000000       2000.000000      78.000000  125000.000000   
50%      3000.000000       2004.000000     110.000000  150000.000000   
75%      6800.000000       2008.000000     143.000000  150000.000000   
max     20000.000000       2016.000000     200.000000  150000.000000   

       RegistrationMonth  NumberOfPictures     PostalCode  
count      317610.000000          317610.0  317610.000000  
mean            5.902799               0.0   50900.158732  
std             3.644733               0.0   25688.278309  
min             0.000000               0.0    1067.000000  
25%             3.000000               0.0   30823.

в результате преобразований датасет сократился на 10 %.  Улучшились показатели зависимости цены (целевого признака) от года регистрации автомобиля и мощности автомобиля.

In [16]:
# выберем из отредактированного датасета целевые признаки для дальнейшего использования в моделях машинного обучения
data_filtered_1 = data_filtered.drop(['DateCrawled',  
                                      'RegistrationMonth', 
                                      'NotRepaired', 
                                      'DateCreated', 
                                      'NumberOfPictures',
                                      'PostalCode',
                                      'LastSeen'], axis=1)
data_filtered_1
data_filtered_2 = data_filtered_1.copy(deep=True)
data_filtered_1

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand
0,480,small,1993,manual,101.0,golf,150000,petrol,volkswagen
2,9800,suv,2004,auto,163.0,grand,125000,petrol,jeep
3,1500,small,2001,manual,75.0,golf,150000,petrol,volkswagen
4,3600,small,2008,manual,69.0,fabia,90000,petrol,skoda
5,650,sedan,1995,manual,102.0,3er,150000,petrol,bmw
...,...,...,...,...,...,...,...,...,...
354362,3200,sedan,2004,manual,122.0,leon,150000,petrol,seat
354363,1150,bus,2000,manual,125.0,zafira,150000,petrol,opel
354366,1199,convertible,2000,auto,101.0,fortwo,125000,petrol,smart
354367,9200,bus,1996,manual,102.0,transporter,150000,petrol,volkswagen


In [17]:
data_filtered_1_ohe = pd.get_dummies(data_filtered_1, drop_first=True)
data_filtered_1_ohe

,Price,RegistrationYear,Power,Kilometer,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,VehicleType_small,VehicleType_suv,...,Brand_saab,Brand_seat,Brand_skoda,Brand_smart,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo
0,480,1993,101.0,150000,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,9800,2004,163.0,125000,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1500,2001,75.0,150000,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,3600,2008,69.0,90000,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
5,650,1995,102.0,150000,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354362,3200,2004,122.0,150000,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
354363,1150,2000,125.0,150000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
354366,1199,2000,101.0,125000,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
354367,9200,1996,102.0,150000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [18]:
# закодируем категориальные целевые признаки
le = LabelEncoder()
data_filtered_1['VehicleType']=le.fit_transform(data_filtered_1['VehicleType'])
data_filtered_1['Gearbox']=le.fit_transform(data_filtered_1['Gearbox'])
data_filtered_1['Model']=le.fit_transform(data_filtered_1['Model'])
data_filtered_1['FuelType']=le.fit_transform(data_filtered_1['FuelType'])
data_filtered_1['Brand']=le.fit_transform(data_filtered_1['Brand'])
data_filtered_1

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand
0,480,5,1993,1,101.0,114,150000,3,37
2,9800,6,2004,0,163.0,115,125000,3,14
3,1500,5,2001,1,75.0,114,150000,3,37
4,3600,5,2008,1,69.0,100,90000,3,31
5,650,4,1995,1,102.0,11,150000,3,2
...,...,...,...,...,...,...,...,...,...
354362,3200,4,2004,1,122.0,138,150000,3,30
354363,1150,0,2000,1,125.0,242,150000,3,24
354366,1199,1,2000,0,101.0,105,125000,3,32
354367,9200,0,1996,1,102.0,217,150000,3,37


In [19]:
features = data_filtered_1.drop(['Price'], axis = 1)
target = data_filtered_1['Price']
features_ohe = data_filtered_1_ohe.drop(['Price'], axis = 1)
target_ohe = data_filtered_1_ohe['Price']
features_lgbm = data_filtered_2.drop(['Price'], axis = 1)
target_lgbm = data_filtered_2['Price']

In [20]:
features_train, features_test, target_train, target_test = train_test_split(features, target, 
                                                                            test_size = 0.2, 
                                                                            random_state=12345)
features_train_ohe, features_test_ohe, target_train_ohe, target_test_ohe = train_test_split(features_ohe, target_ohe, 
                                                                            test_size = 0.2, 
                                                                            random_state=12345)
features_train_lgbm, features_test_lgbm, target_train_lgbm, target_test_lgbm = train_test_split(features_lgbm, target_lgbm, 
                                                                            test_size = 0.2, 
                                                                            random_state=12345)
for c in features_train_lgbm.columns:
    col_type = features_train_lgbm[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        features_train_lgbm[c] = features_train_lgbm[c].astype('category')
        
for c in features_test_lgbm:
    col_type = features_test_lgbm[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        features_test_lgbm[c] = features_test_lgbm[c].astype('category')

Загружен исходный датасет. Исходные данные имеют пропуски в планируемых для использования категориальных целевых признаках. Пропуски заменены подходящими значениями. Нулевые значения в столбцах Price отфильтрованы от нулевых и околонулевых значений и Power взяты аналогично параметрам конкретной модели автомобиля. Категориальные признаки закодированы числами для дальнейшего использования в моделях машинного обучения

## Обучение моделей

### Случайный лес

In [21]:
%%time
score = make_scorer(mean_squared_error, greater_is_better=True)
model_rf = RandomForestRegressor(random_state=12345)
param_set = {
 'n_estimators':[100, 120, 150],
 'max_depth': [10, 12, 15]
}

model_rf = GridSearchCV(model_rf, param_grid = param_set, cv=2, scoring = score)
model_rf.fit(features_train, target_train)

CPU times: user 6min 57s, sys: 1.84 s, total: 6min 59s
Wall time: 6min 59s


GridSearchCV(cv=2, estimator=RandomForestRegressor(random_state=12345),
             param_grid={'max_depth': [10, 12, 15],
                         'n_estimators': [100, 120, 150]},
             scoring=make_scorer(mean_squared_error))

In [22]:
model_rf.best_params_

{'max_depth': 10, 'n_estimators': 100}

In [23]:
model_rf.score(features_train, target_train)**0.5

1978.3824678627623

In [24]:
del features_train
del target_train


### Линейная регрессия

In [25]:
%%time
model_lr = LinearRegression()
model_lr.fit(features_train_ohe, target_train_ohe)

CPU times: user 16.4 s, sys: 22.6 s, total: 39.1 s
Wall time: 39 s


LinearRegression()

In [26]:
score = make_scorer(mean_squared_error, greater_is_better=True)
cv_result = cross_validate(model_lr, features_train_ohe, target_train_ohe, scoring=score, cv=3)
print('RMSE линейная регрессия {:.2f}'.format (abs((cv_result['test_score']**0.5).mean())))

RMSE линейная регрессия 2845.00


### Градиентный бустинг

In [27]:
%%time
score = make_scorer(mean_squared_error, greater_is_better=True)

alg_conf = {
    "num_boost_round":25,
    "max_depth" : 3,
    "num_leaves" : 31,
    'learning_rate' : 0.1,
    'boosting_type' : 'gbdt',
    'objective' : 'regression_l2',
    "early_stopping_rounds": None
}
param_set = {
 'n_estimators':[100, 150],
 'max_depth':[10, 15],
 'learning_rate':[0.05, 0.1]
}

booster = GridSearchCV(estimator = lgbm.LGBMRegressor(random_state = 12345,
                                                     num_leaves=alg_conf["num_leaves"],
                                                     n_estimators=alg_conf["num_boost_round"],
                                                    max_depth=alg_conf["max_depth"],
                                                    learning_rate=alg_conf["learning_rate"],
                                                    objective=alg_conf["objective"],
                                                     ), cv=2, scoring = score, param_grid = param_set)

booster.fit(features_train_lgbm, target_train_lgbm)

CPU times: user 11min 27s, sys: 4.19 s, total: 11min 31s
Wall time: 11min 35s


GridSearchCV(cv=2,
             estimator=LGBMRegressor(max_depth=3, n_estimators=25,
                                     objective='regression_l2',
                                     random_state=12345),
             param_grid={'learning_rate': [0.05, 0.1], 'max_depth': [10, 15],
                         'n_estimators': [100, 150]},
             scoring=make_scorer(mean_squared_error))

In [28]:
booster.score(features_train_lgbm, target_train_lgbm)**0.5

1839.9443941998807

In [29]:
booster.best_params_

{'learning_rate': 0.05, 'max_depth': 10, 'n_estimators': 100}

In [31]:
del features_train_lgbm
del target_train_lgbm

Обучены и валидированы модели случайного леса, линейной регрессии и градиентного бустинга LGBMRegressor. Получены следующие результаты:<br>
Случайный лес: RMSE - 1978 евро,  время обучения с подбором параметров по сетке 7 минут <br>
Линейная регрессия: RMSE - 2845 евро, время обучения 1 минута (параметры по сетке не подбирались)<br>
Градиентный бустинг: RMSE - 1839 евро, время обучения с подбором параметров по сетке 11 минут <br>

## Анализ моделей

### Случайный лес

In [32]:
%%time
model_rf_predict = model_rf.predict(features_test)
print(mean_squared_error(target_test, model_rf_predict, squared=False))

2036.9307149313738
CPU times: user 598 ms, sys: 5.25 ms, total: 604 ms
Wall time: 601 ms


### Линейная регрессия

In [33]:
%%time
model_lr_predict = model_lr.predict(features_test_ohe)
print(mean_squared_error(target_test_ohe, model_lr_predict, squared=False))

2864.4932886613733
CPU times: user 101 ms, sys: 37.9 ms, total: 139 ms
Wall time: 93.1 ms


### Градиентный бустинг

In [34]:
%%time
booster_predict = booster.predict(features_test_lgbm)
print(mean_squared_error(target_test_lgbm, booster_predict, squared=False))

1867.9793128445717
CPU times: user 685 ms, sys: 0 ns, total: 685 ms
Wall time: 697 ms


### Случайный регрессор

In [35]:
%%time
dr = DummyRegressor(strategy='mean')
dr.fit(features_train_ohe, target_train_ohe)
predict_dr = dr.predict(features_test_ohe)
print(mean_squared_error(target_test_ohe, predict_dr, squared=False))

4514.417536013342
CPU times: user 4.17 ms, sys: 0 ns, total: 4.17 ms
Wall time: 2.45 ms


На ранее обученных моделях сделаны предсказания. Получены следующие результаты:<br>
Случайный лес: RMSE - 2037 евро,  время предсказаний 601 милисекунда<br>
Линейная регрессия: RMSE - 2864 евро, время предсказаний 91 милисекунд<br>
Градиентный бустинг: RMSE - 1868 евро, время предсказаний 697 милисекунд <br>
Случайный регрессор: RMSE - 4514 евро, время предсказаний 2,45 милисекунд.
Для дальнейшего использования выберем модель градиентного бустинга как более быструю в обучении и подходящую под заданный критерий ошибки.

Выводы про проекту:<br>
1. Получены входящие данные из базы продаж машин содержащие различные признаки продаваемыех машин. Данные имеют пропуски в категориальных данных и нулевые значения в столбцах с числовыми значениями. Пропущенные категориальные значения заполнены аналогичными параметрами для модели автомобиля. Значения цены отфильтрованы от экстремально низких (нулевых значений). Значения мощности взяты по аналогичными данной модели автомобиля.<br>
2. Обучены модели случайного леса, линейной регрессии и градиентного бустинга. Лучшие результаты по времени обучения и точности предсказаний показала модель градиентного бустинга (RMSE - 1839 евро, время обучения 11 минут).<br>
3. Сделаны предсказания на тестовой выборке моделей. Лучшие показатели времени предсказаний и точности показаны моделью градиентного бустинга (RMSE - 1868 евро, время предсказаний 697 милисекунд). Случайный регрессор показал значения RMSE - 4514 евро, время предсказаний 2,45 милисекунд.<br>
Модель градиентного бустинга лучше остальных показала себя на большом объеме данных по времени обучения/предсказания и точности (ошибке предсказаний).
